# Transform a Fisher matrix to a different parameter set using a Jacobian

In [2]:
import crosspower as cp
import FisherCl as fcl # branch quickCl
import noiseCl as ncl
#import camb

# set plotting to be in the notebook instead of its own qt window
%matplotlib inline

IMPORT ERROR: /opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/quicklens/mpi.pyc (No module named pypar). Could not load pbs or pypar. MPI will not be used.


# 1. Jacobian around CMB-S4 Science Book fiducial parameters

In [2]:
# Need to convert theta_mc to H0
# to get Jacobian:
#   calculate all partial derivatives of cosmomc_theta
# This calculation uses fiducial values to match table 8-1 in CMB-S4 science book

newDeriv = False

AccuracyBoost = 3
nonlinear = True
nz = 300000
nCosParams = 6
paramList = ['ombh2','omch2','H0','As','ns','tau']
deltaP = np.array([0.0008,0.0030,0.1,0.1e-9,0.010,0.020])/2.

cosParams = {
        'H0'    : 69, #setting H0=None allows cosmomc_theta to be used instead
        'cosmomc_theta'           : None,
        'ombh2' : 0.0222,
        'omch2' : 0.120,
        'omk'   : 0,
        'tau'   : 0.06,

        'As'    : 2.20e-9,
        'ns'    : 0.966,
        'r'     : 0,
        'kPivot': 0.05,

        'w'     : -1.0, # actually is w0 but CAMB calls it w
        'wa'    : 0.0,
    
        # if fiducial mnu is changed, need to adjust omch2 as well
        'mnu'   : 0.06, # (eV)
        #'mnu'   : 0.058, # Lloyd suggested this value for fiducial; adjust omch2 if I do use it
        'nnu'   : 3.046,
        'standard_neutrino_neff'  : 3.046,
        'num_massive_neutrinos'   : 1,
        'neutrino_hierarchy'      : 'normal'}

if newDeriv:
    # modified param lists
    myParamsUpper = []
    myParamsLower = []
    for cParamNum in range(nCosParams):
      # add parameter dictionary to lists; HAVE TO BE COPIES!!!
      myParamsUpper.append(cosParams.copy())
      myParamsLower.append(cosParams.copy())
      # modify parameter number cParamNum in ditionaries
      myParamsUpper[cParamNum][paramList[cParamNum]] += deltaP[cParamNum]
      myParamsLower[cParamNum][paramList[cParamNum]] -= deltaP[cParamNum]

    # get MatterPower objs.
    myPksUpper = []
    myPksLower = []
    for cParamNum in range(nCosParams):
        print 'calculating MatterPower objects for ',\
              paramList[cParamNum], ' derivative . . . '

        # create MatterPower objects
        myPksUpper.append(cp.MatterPower(nz=nz,AccuracyBoost=AccuracyBoost,
                             nonlinear=nonlinear,**myParamsUpper[cParamNum]))
        myPksLower.append(cp.MatterPower(nz=nz,AccuracyBoost=AccuracyBoost,
                             nonlinear=nonlinear,**myParamsLower[cParamNum]))



    # ['ombh2','omch2','As','ns','tau']
    dtheta_dombh2 = (myPksUpper[0].cosmomc_theta-myPksLower[0].cosmomc_theta)/(2*deltaP[0])
    dtheta_domch2 = (myPksUpper[1].cosmomc_theta-myPksLower[1].cosmomc_theta)/(2*deltaP[1])
    dtheta_dH0    = (myPksUpper[2].cosmomc_theta-myPksLower[2].cosmomc_theta)/(2*deltaP[2])
    dtheta_dAs    = (myPksUpper[3].cosmomc_theta-myPksLower[3].cosmomc_theta)/(2*deltaP[3])
    dtheta_dns    = (myPksUpper[4].cosmomc_theta-myPksLower[4].cosmomc_theta)/(2*deltaP[4])
    dtheta_dtau   = (myPksUpper[5].cosmomc_theta-myPksLower[5].cosmomc_theta)/(2*deltaP[5])

    dthetas = np.array([dtheta_dombh2,dtheta_domch2,dtheta_dH0,dtheta_dAs,dtheta_dns,dtheta_dtau])

else:
    dthetas = [-0.0248396985185, 0.0104553863834, 2.96219516316e-05, 0.0, 0.0, 0.0]
print dthetas

[-0.0248396985185, 0.0104553863834, 2.96219516316e-05, 0.0, 0.0, 0.0]


In [3]:
# Sometimes the other parts will be needed
# This is a continuation of the previous calculation for mnu and w, though they are not in table 8-1

newDeriv = False

AccuracyBoost = 3
nonlinear = True
nz = 300000
nCosParams = 2
paramList = ['mnu','w']
deltaP = np.array([0.020,0.05])/2.

cosParams = {
        'H0'    : 69, #setting H0=None allows cosmomc_theta to be used instead
        'cosmomc_theta'           : None,
        'ombh2' : 0.0222,
        'omch2' : 0.120,
        'omk'   : 0,
        'tau'   : 0.06,

        'As'    : 2.20e-9,
        'ns'    : 0.966,
        'r'     : 0,
        'kPivot': 0.05,

        'w'     : -1.0, # DARK ENERGY!!!

        # if fiducial mnu is changed, need to adjust omch2 as well
        'mnu'   : 0.06, # (eV)
        #'mnu'   : 0.058, # Lloyd suggested this value for fiducial; adjust omch2 if I do use it
        'nnu'   : 3.046,
        'standard_neutrino_neff'  : 3.046,
        'num_massive_neutrinos'   : 1,
        'neutrino_hierarchy'      : 'normal'}

if newDeriv:
    # modified param lists
    myParamsUpper = []
    myParamsLower = []
    for cParamNum in range(nCosParams):
      # add parameter dictionary to lists; HAVE TO BE COPIES!!!
      myParamsUpper.append(cosParams.copy())
      myParamsLower.append(cosParams.copy())
      # modify parameter number cParamNum in ditionaries
      myParamsUpper[cParamNum][paramList[cParamNum]] += deltaP[cParamNum]
      myParamsLower[cParamNum][paramList[cParamNum]] -= deltaP[cParamNum]

    # get MatterPower objs.
    myPksUpper = []
    myPksLower = []
    for cParamNum in range(nCosParams):
        print 'calculating MatterPower objects for ',\
              paramList[cParamNum], ' derivative . . . '

        # create MatterPower objects
        
        
        # no, don't do it this way for dark energy params.  re-factor to accomodate global variables!
        
        
        myPksUpper.append(cp.MatterPower(nz=nz,AccuracyBoost=AccuracyBoost,
                             nonlinear=nonlinear,**myParamsUpper[cParamNum]))
        myPksLower.append(cp.MatterPower(nz=nz,AccuracyBoost=AccuracyBoost,
                             nonlinear=nonlinear,**myParamsLower[cParamNum]))



    # ['mnu','w']
    dtheta_mnu = (myPksUpper[0].cosmomc_theta-myPksLower[0].cosmomc_theta)/(2*deltaP[0])
    dtheta_w   = (myPksUpper[1].cosmomc_theta-myPksLower[1].cosmomc_theta)/(2*deltaP[1])
    
    dthetas2 = np.array([dtheta_mnu,dtheta_w])

else:
    dthetas2 = [0.00019347, 0.00093481]
print dthetas2

[0.00019347, 0.00093481]


In [4]:
# The only derivatives in the Jacobian are on the dtheta/dparam row.
mcIndex = 2
jacobian = np.diag(np.ones(6)) # 6 is nCosParams above
jacobian[mcIndex,:] = dthetas#*100
#jacobian[:,mcIndex] = dthetas#*100

# multiply these all by d{100thetamc}/d{thetamc} = 100
jacobian[mcIndex,:] *= 100
#jacobian[:,mcIndex] *= 100

print jacobian

[[ 1.          0.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.          0.        ]
 [-2.48396985  1.04553864  0.0029622   0.          0.          0.        ]
 [ 0.          0.          0.          1.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          0.        ]
 [ 0.          0.          0.          0.          0.          1.        ]]


In [5]:
# jacobian-it-up
#invCovPlanck = np.dot(jacobian.T,np.dot(invCovPlanck,jacobian))
#print invCovPlanck

NameError: name 'invCovPlanck' is not defined

In [ ]:
# ok, also convert ln(10^10*As) to 10^9*As
# exponentiate and divide by 10?  No, use Jacobian again.
# no, actually convert to just As, since that's what FijTE uses
As_fid = 2.2e-9
AsIndex = 3
invCovPlanck[AsIndex,:] *= 1./As_fid
invCovPlanck[:,AsIndex] *= 1./As_fid

# 2. Jacobian around my FisherCl fiducial parameters

In [3]:
# convert cosmomc_theta to H_0
# The fiducial parameters list is the default from FisherCl, 
#   with cosmomc_theta switched to None and H0 switched to a typical FisherCl value of 67.49

newDeriv = True# False

AccuracyBoost = 3
nonlinear = False #True
nz = 300000
#nCosParams = 8
#paramList =       ['ombh2','omch2','H0','As'  ,'ns' ,'tau','mnu','w' ]
#deltaP = np.array([0.0008 ,0.0030 ,0.1 ,0.1e-9,0.010,0.020,0.020,0.05])/2.
nCosParams = 9 #8
paramList =       ['ombh2','omch2','H0','As'  ,'ns' ,'tau','mnu','w' ,'wa' ]
deltaP = np.array([0.0008 ,0.0030 ,0.1 ,0.1e-9,0.010,0.020,0.020,0.05,0.025])/2.

cosParams = {
    'H0'    : 67.49, #None, #67.51, #setting H0=None allows cosmomc_theta to be used instead
    'cosmomc_theta' : None, #1.04087e-2,
    'ombh2' : 0.02226,
    'omch2' : 0.1193,
    'omk'   : 0,
    'tau'   : 0.063,

    'As'    : 2.130e-9,
    'ns'    : 0.9653,
    'r'     : 0,
    'kPivot': 0.05,

    'w'     : -1.0,
    'wa'    : 0.0, # new param!

    # if fiducial mnu is changed, need to adjust omch2 as well
    'mnu'   : 0.06, # (eV)
    #'mnu'   : 0.058, # Lloyd suggested this value for fiducial; adjust omch2 if I do use it
    'nnu'   : 3.046,
    'standard_neutrino_neff'  : 3.046,
    'num_massive_neutrinos'   : 1,
    'neutrino_hierarchy'      : 'normal'}

if newDeriv:
    # modified param lists
    myParamsUpper = []
    myParamsLower = []
    for cParamNum in range(nCosParams):
      # add parameter dictionary to lists; HAVE TO BE COPIES!!!
      myParamsUpper.append(cosParams.copy())
      myParamsLower.append(cosParams.copy())
      # modify parameter number cParamNum in ditionaries
      myParamsUpper[cParamNum][paramList[cParamNum]] += deltaP[cParamNum]
      myParamsLower[cParamNum][paramList[cParamNum]] -= deltaP[cParamNum]

    # get MatterPower objs.
    #myPksUpper = []
    #myPksLower = []
    cosmomc_thetas_upper = []
    cosmomc_thetas_lower = []
    for cParamNum in range(nCosParams):
        print 'calculating MatterPower objects for ',\
              paramList[cParamNum], ' derivative . . . '

        # create MatterPower objects
        
        
        # no, don't do it this way for dark energy params.  re-factor to accomodate global variables!
        #myPksUpper.append(cp.MatterPower(nz=nz,AccuracyBoost=AccuracyBoost,
        #                     nonlinear=nonlinear,**myParamsUpper[cParamNum]))
        #myPksLower.append(cp.MatterPower(nz=nz,AccuracyBoost=AccuracyBoost,
        #                     nonlinear=nonlinear,**myParamsLower[cParamNum]))
        
        myPkUpper = cp.MatterPower(nz=nz,AccuracyBoost=AccuracyBoost,
                                   nonlinear=nonlinear,**myParamsUpper[cParamNum])
        cosmomc_thetas_upper.append(myPkUpper.cosmomc_theta)
        
        myPkLower = cp.MatterPower(nz=nz,AccuracyBoost=AccuracyBoost,
                                   nonlinear=nonlinear,**myParamsLower[cParamNum])
        cosmomc_thetas_lower.append(myPkLower.cosmomc_theta)
        
    dthetas = (np.array(cosmomc_thetas_upper)-np.array(cosmomc_thetas_lower))/(2*deltaP)

    #dtheta_dombh2 = (myPksUpper[0].cosmomc_theta-myPksLower[0].cosmomc_theta)/(2*deltaP[0])
    #dtheta_domch2 = (myPksUpper[1].cosmomc_theta-myPksLower[1].cosmomc_theta)/(2*deltaP[1])
    #dtheta_dH0    = (myPksUpper[2].cosmomc_theta-myPksLower[2].cosmomc_theta)/(2*deltaP[2])
    #dtheta_dAs    = (myPksUpper[3].cosmomc_theta-myPksLower[3].cosmomc_theta)/(2*deltaP[3])
    #dtheta_dns    = (myPksUpper[4].cosmomc_theta-myPksLower[4].cosmomc_theta)/(2*deltaP[4])
    #dtheta_dtau   = (myPksUpper[5].cosmomc_theta-myPksLower[5].cosmomc_theta)/(2*deltaP[5])
    #dtheta_mnu    = (myPksUpper[6].cosmomc_theta-myPksLower[6].cosmomc_theta)/(2*deltaP[6])
    #dtheta_w      = (myPksUpper[7].cosmomc_theta-myPksLower[7].cosmomc_theta)/(2*deltaP[7])
    #dtheta_wa     = (myPksUpper[8].cosmomc_theta-myPksLower[8].cosmomc_theta)/(2*deltaP[8])
    
    #print dtheta_dombh2,dtheta_domch2,dtheta_dH0,dtheta_dAs,dtheta_dns,dtheta_dtau,dtheta_mnu,dtheta_w
    #dthetas = np.array([dtheta_dombh2,dtheta_domch2,dtheta_dH0,dtheta_dAs,dtheta_dns,dtheta_dtau,dtheta_mnu,dtheta_w])
    
    #print dtheta_dombh2,dtheta_domch2,dtheta_dH0,dtheta_dAs,dtheta_dns,dtheta_dtau,dtheta_mnu,dtheta_w,dtheta_wa
    #dthetas = np.array([dtheta_dombh2,dtheta_domch2,dtheta_dH0,dtheta_dAs,dtheta_dns,dtheta_dtau,dtheta_mnu,
    #                    dtheta_w,dtheta_wa])

else:
    dthetas = [-2.46099289e-02,  1.05373002e-02,  2.98929388e-05,  # dtheta_dombh2, dtheta_domch2, dtheta_dH0
               0.00000000e+00,  0.00000000e+00,  0.00000000e+00,   # dtheta_dAs, dtheta_dns, dtheta_dtau
               1.94175593e-04,  8.94926461e-04,  2.43189330e-04]   # dtheta_mnu, dtheta_w, dtheta_wa


print dthetas

calculating MatterPower objects for  ombh2  derivative . . . 
neutrino_hierarchy =  normal
starting makePkInterp.


finishing makePkInterp.
neutrino_hierarchy =  normal
starting makePkInterp.


finishing makePkInterp.
calculating MatterPower objects for  omch2  derivative . . . 
neutrino_hierarchy =  normal
starting makePkInterp.


finishing makePkInterp.
neutrino_hierarchy =  normal
starting makePkInterp.


finishing makePkInterp.
calculating MatterPower objects for  H0  derivative . . . 
neutrino_hierarchy =  normal
starting makePkInterp.


...

finishing makePkInterp.
calculating MatterPower objects for  w  derivative . . . 
neutrino_hierarchy =  normal
starting makePkInterp.


finishing makePkInterp.
neutrino_hierarchy =  normal
starting makePkInterp.


finishing makePkInterp.
calculating MatterPower objects for  wa  derivative . . . 
neutrino_hierarchy =  normal
starting makePkInterp.


finishing makePkInterp.
neutrino_hierarchy =  normal
starting makePkInterp.
finishing makePkInterp.


TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [4]:
dthetas = (np.array(cosmomc_thetas_upper)-np.array(cosmomc_thetas_lower))/(2*deltaP)
print dthetas

[-2.46099289e-02  1.05373002e-02  2.98929388e-05  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.94175593e-04  8.94926461e-04
  2.43189330e-04]
